In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import cv2
from sklearn.preprocessing import LabelEncoder

In [3]:
dataset = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

In [4]:
lbEncode = LabelEncoder()

In [5]:
dataset['labels'] = lbEncode.fit_transform(dataset['labels'])

In [6]:
PATH = "../input/plant-pathology-2021-fgvc8/test_images/"

In [7]:
def load_image(image_id):
    file_path = str(image_id)
    img = cv2.imread(PATH+file_path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
class bottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_planes, planes, stride=1, dim_change=None):
        super(bottleNeck, self).__init__()
        
        self.conv1 = nn.Conv2d(in_planes, planes, stride=1, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, stride=stride, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes*self.expansion, kernel_size=1)
        self.bn3 = nn.BatchNorm2d(planes*self.expansion)
        self.dim_change = dim_change
    
    def forward(self, x):
        res = x
        
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        
        if self.dim_change is not None:
            res = self.dim_change(res)
            
        out += res
        out = F.relu(out)
        
        return out

In [10]:
class baseBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1, dim_change=None):
        super(baseBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_planes, planes, stride=stride, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, stride=1, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)
        self.dim_change = dim_change
    def forward(self, x):
        res = x
        
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        
        if self.dim_change is not None:
            res = self.dim_change(res)
        
        out += res
        out = F.relu(out)
        
        return out

In [11]:
class ResNet(nn.Module):
    def __init__(self, block, num_layers, num_classes=12):
        super(ResNet, self).__init__()
        
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._layer(block, 64, num_layers[0], stride=1)
        self.layer2 = self._layer(block, 128, num_layers[1], stride=2)
        self.layer3 = self._layer(block, 256, num_layers[2], stride=2)
        self.layer4 = self._layer(block, 512, num_layers[3], stride=2)
        self.avgPool = nn.AvgPool2d(kernel_size=4, stride=1)
        self.fc = nn.Linear(25088, num_classes)
        
    def _layer(self, block, planes, num_layers, stride=1):
        dim_change = None
        if stride != 1 or planes != self.in_planes * block.expansion:
            dim_change = nn.Sequential(nn.Conv2d(self.in_planes, planes*block.expansion, kernel_size=1, stride=stride),
                                       nn.BatchNorm2d(planes*block.expansion))
        netLayers = []
        netLayers.append(block(self.in_planes, planes, stride=stride, dim_change=dim_change))
        self.in_planes = planes * block.expansion
        for i in range(1, num_layers):
            netLayers.append(block(self.in_planes, planes))
            self.in_planes = planes * block.expansion
            
        return nn.Sequential(*netLayers)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = F.avg_pool2d(x, 4)
        x = x.flatten(1)
        x = self.fc(x)
        
        return x

In [12]:
model = ResNet(baseBlock, [3, 4, 6, 3]).to(device)
model.load_state_dict(torch.load('../input/plant-pathology/model.pth'))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): baseBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): baseBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): baseBlock(
      (conv1): Conv2d(64, 64, kerne

In [13]:
def preprocess(img):
    image = load_image(img)
    image = cv2.resize(image, (224, 224))
    image = torch.tensor(image, dtype=torch.float32) / 255
    image = image.reshape(3, 224, 224)
    
    return image

In [14]:
class Data(Dataset):
    def __init__(self, X, y=None, transform=None):
        self.image_id = X
        self.labels = y
        self.transforms = transform
    
    def __len__(self):
        return len(self.image_id)
    
    def __getitem__(self, idx):
        if self.labels is not None:
            image_id = self.image_id[idx][0]
            label = self.labels[idx]
        
            image = preprocess(image_id)
            if self.transforms:
                image = self.transforms(image)
        
            return image, label
        else:
            image_id = self.image_id[idx][0]
        
            image = preprocess(image_id)
            if self.transforms:
                image = self.transforms(image)
            
            return image

In [15]:
def predict(data_load):
    test_pred = torch.LongTensor()
    
    for i, image in enumerate(data_load):
        image = image.to(device)
        
        output = model(image)
        
        pred = output.cpu().max(1, keepdim=True)[1]
        test_pred = torch.cat((test_pred, pred), dim=0)
        
    return test_pred

In [16]:
test_dataset = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
test_dataset = test_dataset.drop(['labels'], axis=1)

In [17]:
test_data = Data(test_dataset.values)
test_loader = DataLoader(test_data, batch_size=4)

In [18]:
predictions = predict(test_loader)
predictions

tensor([[6],
        [0],
        [9]])

In [19]:
predict_df = pd.DataFrame(lbEncode.inverse_transform(predictions.numpy().ravel()))

In [20]:
predict_df = pd.concat([test_dataset, predict_df], ignore_index=True, axis=1)
predict_df = predict_df.rename(columns={predict_df.columns[0]: 'image', predict_df.columns[1]: 'labels'})

In [21]:
predict_df.to_csv('submission.csv', index=False)